### Installing of Requirements (only run this if you haven't installed the necessary libraries found in requirements.txt)

In [1]:
# !pip install -r requirements.txt

In [3]:
import geopandas as  gpd
import os
import pandas as pd

Todo
- convert all geojson files into 1 geodataframe 
- data cleaning
- clip the inputs onto the map layer, similar to getting the intersection between the points and the map layer
- return the output onto a file
- write documentation in this notebook if necessary

### Reading the Paranaque Images and Saving a compiled GEOJSON file of the images 

In [2]:
# Accessing the data of Paranque images
paranaque_json_dir = 'data/paranaque'

os.chdir(paranaque_json_dir)

In [3]:
gdf_list = []
geojson_Count = 0

In [4]:
for file in os.listdir():
    if file.endswith('.geojson'):
        gdf = gpd.read_file(file)
        geojson_Count += 1
        
        # filter out null json files
        if '"features": []' not in gdf:
            gdf = gpd.read_file(file)
            gdf_list.append(gdf)
            
            
        # gdf_list.append(gdf)

In [6]:
# Run this if you want to view the compiled json

# gdf_list

### Saving the compiled images into a GeoDataFrame in a GEOJSON file

In [7]:
# output_dir = 'output_geojson'
# os.makedirs(output_dir, exist_ok=True)

# merged_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)
# output_filename = 'output_geojson/merged_paranaque.geojson'
# merged_gdf.to_file(output_filename, driver='GeoJSON')
# print(f"Saved {output_filename}")

#the file is found in a folder that can be viewed in the data/paranaque folder as "output_geojson"

In [8]:
# %pip install imagehash

In [5]:
import os
import pandas as pd
import geopandas as gpd
from PIL import Image
import imagehash

def hash_image(file_path):
    """
    Compute a hash for an image file.
    """
    with Image.open(file_path) as img:
        return imagehash.average_hash(img)

def filter_similar_images(gdf_list):
    """
    Filter out similar images based on their hash.
    """
    filtered_gdf_list = []
    hash_set = set()

    for gdf in gdf_list:
        file_path = gdf['thumb_2048_url']  # Replace 'your_image_column' with the column containing image file paths
        file_hash = hash_image(file_path)

        if file_hash not in hash_set:
            hash_set.add(file_hash)
            filtered_gdf_list.append(gdf)

    return filtered_gdf_list

In [7]:
# %pip install tensorflow

## Code for setting up model

In [5]:
import requests
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from io import BytesIO

# Define the model
def build_siamese_network(input_shape):
    left_input = layers.Input(shape=input_shape)
    right_input = layers.Input(shape=input_shape)

    # Shared convolutional layers
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))

    # Encoding for both left and right inputs
    encoded_left = model(left_input)
    encoded_right = model(right_input)

    # L1 distance layer between the two encoded representations
    L1_layer = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))

    # Add the distance layer to the network
    L1_distance = L1_layer([encoded_left, encoded_right])

    # Prediction layer
    prediction = layers.Dense(1, activation='sigmoid')(L1_distance)

    # Create and compile the model
    siamese_model = models.Model(inputs=[left_input, right_input], outputs=prediction)
    siamese_model.compile(loss=BinaryCrossentropy(), optimizer=Adam(), metrics=['accuracy'])

    return siamese_model

# Function to load and preprocess images from URL
def preprocess_image_from_url(image_url, target_size):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = tf.cast(img_array, tf.float32) / 255.0  # Normalize pixel values to be between 0 and 1
    return img_array

# Example usage
image1_url = "https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t6/An9cxUlW_AayC--U1c1tXtigRWBRepZES4Saq31GZPPfI_1kASse71rEi1aHOXJg7PgemaXitnPVZj7-Gu3DCgPYpqEpwxl8xyqfFQExorMuMYfThwfTk1o7hkNvHVDmUNNzmOxxMihFXrV1DWZbUQ?stp=s2048x1152&ccb=10-5&oh=00_AfABtAA7BtX7l74UEQ9ZLtj1PaNJfoLMuund6A1JT3mVIQ&oe=65B3421E&_nc_sid=201bca"
image2_url = "https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t6/An_KYEqGfSXn0q4WRrqa5yKMBoBGH2ASguDbtT_RWVlbONyd3wa2GQn_D2n5o1hEzzh3HJjCCRpBnakmbUbOvOxrjKhCTQD0j14SK0uibENIo5H0O_Ma6KpP2rF1TbgEreu3R2Ukib0Vr9bJds2L-g?stp=s2048x1152&ccb=10-5&oh=00_AfBb33hgHaxhlRnrovVaeukq6ZdKdcY_6Meb-OI7UGSYzQ&oe=65B33919&_nc_sid=201bca"
image3_url = "https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t6/An9WdAlvQB4b3vVt8c0cKC5EvIAPus33Q0BT3_N-7Q7L4EFNsE0RNbe58SZmbc6tPZyUszp_UHNydb-UHZgf-kcHnpzU2Nma4bURfjzUPSQixw4NPeUU5fqN6LktzzsIX9rYg7GAOg4f6yAVPCBbnA?stp=s2048x1152&ccb=10-5&oh=00_AfA7aberfS7qHOSy63gM3E_QZqwdRSk74GbNmagfB2O3YQ&oe=65B34BA0&_nc_sid=201bca"
image4_url = "https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t6/An-0tQBuUEa8IDZHhx03VckPyeF658vlfksT9BcLeybQ1GtmbFVQlD6O0y3oPx4ehZV5PFiFs63E9Y48CCTirCw1Be6vT2cLzsi-jKYLwOZ2fWm94_V1irwiyEWTsd03qRVvu3ugOoa72YF9V_cHDg?stp=s2048x1536&ccb=10-5&oh=00_AfDfkLUS9BEDGUfpOkuQQp_Jx-fLhpoUROJVJuaU0XfXHw&oe=65B32AC6&_nc_sid=201bca"

target_size = (128, 128)  # Adjust the target size as needed

image1 = preprocess_image_from_url(image1_url, target_size)
image2 = preprocess_image_from_url(image2_url, target_size)
image3 = preprocess_image_from_url(image3_url, target_size)
image4 = preprocess_image_from_url(image4_url, target_size)

input_shape = image1.shape[1:]  # Shape of the preprocessed image

# Combine pairs and labels into a list for training
image_pairs = ([image1, image2], [image1, image4])

# Separate the pairs into left and right inputs
left_inputs = np.array(image_pairs)[:, 0, :, :, :]
right_inputs = np.array(image_pairs)[:, 1, :, :, :]

# Assuming you have labels indicating whether the pairs are similar or not
labels = np.array([1, 0])  # 1 if similar, 0 if not

# Reshape inputs to match the model's expectations
# take note of this
left_inputs = left_inputs.reshape((-1,) + left_inputs.shape[2:])
right_inputs = right_inputs.reshape((-1,) + right_inputs.shape[2:])

siamese_model = build_siamese_network(input_shape)
siamese_model.summary()

siamese_model.fit([left_inputs, right_inputs], labels, epochs=10, batch_size=2)


C:\Users\Sean\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 128)                  7392320   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 lambda (Lambda)             (None, 128)                  0         ['sequential[0][0]',   

## Code for checking similarity score of images

In [24]:
# Assuming you have a trained Siamese model named 'siamese_model'

# Load and preprocess two new images from URLs
new_image1_url = "https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t6/An9WdAlvQB4b3vVt8c0cKC5EvIAPus33Q0BT3_N-7Q7L4EFNsE0RNbe58SZmbc6tPZyUszp_UHNydb-UHZgf-kcHnpzU2Nma4bURfjzUPSQixw4NPeUU5fqN6LktzzsIX9rYg7GAOg4f6yAVPCBbnA?stp=s2048x1152&ccb=10-5&oh=00_AfA7aberfS7qHOSy63gM3E_QZqwdRSk74GbNmagfB2O3YQ&oe=65B34BA0&_nc_sid=201bca"
new_image2_url = "https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t6/An9WdAlvQB4b3vVt8c0cKC5EvIAPus33Q0BT3_N-7Q7L4EFNsE0RNbe58SZmbc6tPZyUszp_UHNydb-UHZgf-kcHnpzU2Nma4bURfjzUPSQixw4NPeUU5fqN6LktzzsIX9rYg7GAOg4f6yAVPCBbnA?stp=s2048x1152&ccb=10-5&oh=00_AfA7aberfS7qHOSy63gM3E_QZqwdRSk74GbNmagfB2O3YQ&oe=65B34BA0&_nc_sid=201bca"

new_image1 = preprocess_image_from_url(new_image1_url, target_size)
new_image2 = preprocess_image_from_url(new_image2_url, target_size)

# Make predictions using the trained model
predictions = siamese_model.predict([new_image1, new_image2])

# Interpret the predictions
similarity_probability = predictions[0][0]

similarity_probability
# # Threshold for similarity (you can adjust this based on your needs)
# similarity_threshold = 0.5

# if similarity_probability >= similarity_threshold:
#     print("The images are similar.")
# else:
#     print("The images are dissimilar.")


1/1 [==============================] - 0s 22ms/step


0.5002883

In [26]:
os.getcwd()

'C:\\Users\\Sean\\Documents\\Python Notebooks\\street-image-extractor'

In [22]:
os.chdir('..')

# MODEL TESTING

In [2]:
img_df = pd.read_csv("train.csv")
img_df

,img1,img2,label
0,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
1,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
2,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
3,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
4,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
...,...,...,...
133,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,0
134,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,0
135,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
136,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1


In [3]:
# img1 = img_df['img1'].tolist()
# img2 = img_df['img2'].tolist()

# labels = img_df['label'].tolist()


In [18]:
img1 = [preprocess_image_from_url(row, target_size) for row in img_df['img1'].tolist()]
img2 = [preprocess_image_from_url(row, target_size) for row in img_df['img2'].tolist()]

labels = img_df['label'].tolist()

In [19]:
target_size = (128, 128)  # Adjust the target size as needed

input_shape = (138, 138, 3)

In [11]:
# Resize images to the desired shape (138, 138, 3) and add padding if necessary
img1_resized = [tf.image.resize_with_pad(image, 138, 138) for image in img1]
img2_resized = [tf.image.resize_with_pad(image, 138, 138) for image in img2]

# Convert lists of resized images to NumPy arrays
img1 = np.array([np.expand_dims(image, axis=0) for image in img1_resized])
img2 = np.array([np.expand_dims(image, axis=0) for image in img2_resized])

labels = np.array(labels)

print(img1.shape, img2.shape, labels.shape)

(138, 1, 1, 138, 138, 3) (138, 1, 1, 138, 138, 3) (138,)


In [21]:
img1 = np.array([preprocess_image_from_url(row, target_size) for row in img_df['img1'].tolist()])
img2 = np.array([preprocess_image_from_url(row, target_size) for row in img_df['img2'].tolist()])

labels = np.array(img_df['label'].tolist())

img1 = img1.reshape((-1,) + img1.shape[2:])
img2 = img2.reshape((-1,) + img2.shape[2:])


siamese_model.fit([img1, img2], labels, epochs=100, batch_size=2)

Epoch 1/100
56/69 [=======================>......] - ETA: 0s

ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [22]:
# print("img1 shape:", img1.shape)
# print("img2 shape:", img2.shape)
# print("labels shape:", labels.shape)


img1 shape: (138, 128, 128, 3)
img2 shape: (138, 128, 128, 3)
labels shape: (138,)


In [15]:
siamese_model = build_siamese_network(input_shape)
siamese_model.summary()

# Train the model
siamese_model.fit([img1, img2], labels, epochs=100, batch_size=2)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 138, 138, 3)]        0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 138, 138, 3)]        0         []                            
                                                                                                  
 sequential_3 (Sequential)   (None, 128)                  8940608   ['input_7[0][0]',             
                                                                     'input_8[0][0]']             
                                                                                                  
 lambda_3 (Lambda)           (None, 128)                  0         ['sequential_3[0][0]',  

ValueError: in user code:

    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    Va

In [ ]:
# for row in img1:
#     # image = preprocess_image_from_url(row, target_size)

In [ ]:
# for row in img2:
#     image = preprocess_image_from_url(row, target_size)

In [71]:

# # Convert lists to NumPy arrays
# img1 = np.array(img1)
# img2 = np.array(img2)

# # Assuming 'labels' is also a list, convert it to a NumPy array
# labels = np.array(labels)

# siamese_model = build_siamese_network(input_shape)
# siamese_model.summary()

# siamese_model.fit([img1, img2], labels, epochs=100, batch_size=2)


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_25 (InputLayer)       [(None, 138, 138, 3)]        0         []                            
                                                                                                  
 input_26 (InputLayer)       [(None, 138, 138, 3)]        0         []                            
                                                                                                  
 sequential_12 (Sequential)  (None, 128)                  8940608   ['input_25[0][0]',            
                                                                     'input_26[0][0]']            
                                                                                                  
 lambda_12 (Lambda)          (None, 128)                  0         ['sequential_12[0][0]',

ValueError: in user code:

    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Sean\anaconda3\lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    Va

In [67]:
# print(img1.shape, img2.shape, labels.shape)

(138,) (138,) (138,)


In [ ]:
# filtered_gdf_list = filter_similar_images(gdf_list)

# output_dir = 'output_geojson'
# os.makedirs(output_dir, exist_ok=True)

# merged_gdf = gpd.GeoDataFrame(pd.concat(filtered_gdf_list, ignore_index=True), crs=filtered_gdf_list[0].crs)
# output_filename = 'output_geojson/merged_paranaque.geojson'
# merged_gdf.to_file(output_filename, driver='GeoJSON')
# print(f"Saved {output_filename}")


AttributeError: 'Series' object has no attribute 'read'

# NEW MODEL ATTEMPT

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Function to create the base convolutional neural network (CNN) for image embedding
def create_base_network(input_shape):
    input_layer = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(input_layer)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    model = Model(inputs=input_layer, outputs=x)
    return model

# Function to calculate the Euclidean distance between two image embeddings
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_squared = K.sum(K.square(vector1 - vector2), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Function to create the Siamese network
def create_siamese_network(input_shape):
    input_image1 = Input(shape=input_shape)
    input_image2 = Input(shape=input_shape)

    base_network = create_base_network(input_shape)

    # Getting the encoded representation of the input images
    encoded_image1 = base_network(input_image1)
    encoded_image2 = base_network(input_image2)

    # Calculating the distance between the encoded representations
    distance = Lambda(euclidean_distance)([encoded_image1, encoded_image2])

    # Creating the Siamese model
    siamese_model = Model(inputs=[input_image1, input_image2], outputs=distance)

    return siamese_model




C:\Users\Sean\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
img_df = pd.read_csv("train.csv")
img_df

,img1,img2,label
0,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
1,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
2,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
3,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
4,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
...,...,...,...
133,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,0
134,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,0
135,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1
136,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,https://scontent.fmnl30-2.fna.fbcdn.net/m1/v/t...,1


In [20]:
import requests
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from io import BytesIO

# Define the model
def build_siamese_network(input_shape):
    left_input = layers.Input(shape=input_shape)
    right_input = layers.Input(shape=input_shape)

    # Shared convolutional layers
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))

    # Encoding for both left and right inputs
    encoded_left = model(left_input)
    encoded_right = model(right_input)

    # L1 distance layer between the two encoded representations
    L1_layer = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))

    # Add the distance layer to the network
    L1_distance = L1_layer([encoded_left, encoded_right])

    # Prediction layer
    prediction = layers.Dense(1, activation='sigmoid')(L1_distance)

    # Create and compile the model
    siamese_model = models.Model(inputs=[left_input, right_input], outputs=prediction)
    siamese_model.compile(loss=BinaryCrossentropy(), optimizer=Adam(), metrics=['accuracy'])

    return siamese_model

def preprocess_image_from_url(image_url, target_size):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = tf.cast(img_array, tf.float32) / 255.0  # Normalize pixel values to be between 0 and 1
    return img_array


In [59]:
input_shape = (205, 205, 3)  # Adjust input shape based on your images
target_size = (205, 205) 

In [60]:

img1 = np.array([preprocess_image_from_url(row, target_size) for row in img_df['img1'].tolist()])
img2 = np.array([preprocess_image_from_url(row, target_size) for row in img_df['img2'].tolist()])

labels = np.array(img_df['label'].tolist())

img1 = img1.reshape((-1,) + img1.shape[2:])
img2 = img2.reshape((-1,) + img2.shape[2:])


In [61]:
# Compile the model
siamese_model = build_siamese_network(input_shape)

# siamese_model = create_siamese_network(input_shape)
siamese_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

siamese_model.summary()


# Training the Siamese network with your image pairs and similarity scores
# X_train_image1, X_train_image2 are the image pairs, and y_train is the similarity score (1 or 0)
siamese_model.fit([img1, img2], labels, epochs=100, batch_size=32)

# After training, you can use the model to predict the similarity between two new images
# similarity_score = siamese_model.predict([image1, image2])

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 205, 205, 3)]        0         []                            
                                                                                                  
 input_17 (InputLayer)       [(None, 205, 205, 3)]        0         []                            
                                                                                                  
 sequential_3 (Sequential)   (None, 128)                  1968851   ['input_16[0][0]',            
                                                          2          'input_17[0][0]']            
                                                                                                  
 lambda_6 (Lambda)           (None, 128)                  0         ['sequential_3[0][0]',  

In [68]:
# Assuming you have a trained Siamese model named 'siamese_model'

# Load and preprocess two new images from URLs
new_image1_url = "https://scontent.fmnl17-5.fna.fbcdn.net/m1/v/t6/An_lvFyyeHWaA-AbolXOaEV15UuxBfj47oSxTDm4DphxMJiTOmQmiprhxavhYwQj4HFFQMKyDm5xtaCFAwFP_kcu1v9CKYZpWkESkmfj_mpPIu5zFKy5vHPHhzfY1wfqWzexos_Gctg_mUR1pbdG2MM?stp=s2048x1152&ccb=10-5&oh=00_AfDeBrhFbPRRxiHa37hIMGrfiSke2tCkuDkeYMWwTEEUAg&oe=65CEC08E&_nc_sid=201bca"
new_image2_url = "https://scontent.fmnl17-5.fna.fbcdn.net/m1/v/t6/An8l2r73oTUMYSbZkOogu4jRHbAFQOxAJGM7gtKWVNpPLnOkYMG8RJXmhK4mkzhz9pn4HjwMfM01HDgFns5bmXyc_vQ0sAI9IPMaO8-Q9j3DwB9YR0yZDtHGfoMlEin4nMXyuDKzQxAijP1KyzWNGM4?stp=s2048x1152&ccb=10-5&oh=00_AfACjdBjotxJCVGcOvr8rlfyVfkGg4Qw5KHZpiUZcriDFQ&oe=65CEA448&_nc_sid=201bca"

new_image1 = preprocess_image_from_url(new_image1_url, target_size)
new_image2 = preprocess_image_from_url(new_image2_url, target_size)

# Make predictions using the trained model
predictions = siamese_model.predict([new_image1, new_image2])

# Interpret the predictions
similarity_probability = predictions[0][0]

print(similarity_probability)

# # Threshold for similarity (you can adjust this based on your needs)
similarity_threshold = 0.001

if similarity_probability >= similarity_threshold:
    print("The images are similar.")
else:
    print("The images are dissimilar.")


1/1 [==============================] - 0s 23ms/step
0.041213818
The images are similar.
